In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import os
import re
import numpy as np
import pandas as pd
import collections

In [2]:
x_name = os.path.join(os.getcwd(), '데이터SET', f"{'[Track1_데이터3] samp_cst_feat'}.csv")
df_x = pd.read_csv(x_name, index_col=0)

y_name = os.path.join(os.getcwd(), '데이터SET', f"{'[Track1_데이터2] samp_train'}.csv")
df_y = pd.read_csv(y_name, index_col=0)

df = pd.merge(df_x, df_y, on='cst_id_di')

In [3]:
df.MRC_ID_DI[df.MRC_ID_DI > 0 ] = 1

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [4]:
nc_name = os.path.join(os.getcwd(), '데이터SET', f"{'[Track1_데이터4] variable_dtype'}.xlsx")
nc = pd.read_excel(nc_name, index_col=0)

In [5]:
from  sklearn.preprocessing import scale
from statsmodels.nonparametric.kde import KDEUnivariate

def find_outliers_kde(x):
    x_scaled = scale(list(map(float, x)))
    kde = KDEUnivariate(x_scaled)
    kde.fit(bw='scott', fft=True)
    pred = kde.evaluate(x_scaled)
    
    n = sum(pred < 0.001)
    outlier_ind = np.asarray(pred).argsort()[:n]
    outlier_value = np.asarray(x)[outlier_ind]
    
    return outlier_ind, outlier_value

for i in df.columns[0:-1]:
    if nc.loc[i, 'dType'] == 'numerical':
        kde_indices, kde_values = find_outliers_kde(df[i])
        df_01 = df[i].quantile(0.01)
        df_99 = df[i].quantile(0.99)
        df_50 = df[i].quantile(0.50)
        for j in range(len(df)):
            if df[i].values[j] in kde_values:
                df.at[df.index[j], i] = df_50
"""
for i in df.columns[0:-1]:
    if nc.loc[i, 'dType'] == 'numerical':
        d_90 = df[i].quantile(0.99)
        d_10 = df[i].quantile(0.01)
        d_50 = df[i].quantile(0.50)
        df[i] = np.where(df[i] > d_90, d_90, df[i])
        df[i] = np.where(df[i] < d_10, d_10, df[i])
"""

print(df.shape)
#(10124, 227)

(10124, 227)


In [6]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import random

physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

seed_value= 0
os.environ['PYTHONHASHSEED']=str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

In [7]:
import sys
ipython = get_ipython()

def hide_traceback(exc_tuple=None, filename=None, tb_offset=None,
                      exception_only=False, running_compiled_code=False):
       etype, value, tb = sys.exc_info()
       return ipython._showtraceback(etype, value, ipython.InteractiveTB.get_exception_only(etype, value))

ipython.showtraceback = hide_traceback

In [8]:
df_0 = df[df['MRC_ID_DI'] == 0].sample(frac=1)
df_1 = df[df['MRC_ID_DI'] == 1].sample(frac=1)

sample_size = len(df_0) if len(df_0) < len(df_1) else len(df_1)

df_h = pd.concat([df_0.head(sample_size), df_1.head(sample_size)]).sample(frac=1)
df_t = pd.concat([df_0.tail(sample_size), df_1.head(sample_size)]).sample(frac=1)
df_f = pd.concat([df_h, df_t]).sample(frac=1)

In [9]:
X = df_f.drop(columns = ['MRC_ID_DI'], axis=1)
#X = df_f.drop(columns = ['MRC_ID_DI', 'VAR021', 'VAR046'], axis=1)
#y = tf.keras.utils.to_categorical(df_f['MRC_ID_DI'])
y = df_f['MRC_ID_DI']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [11]:
num, cat = list(), list()
for i in X.columns:
    if nc.loc[i, 'dType'] == 'numerical':
        num.append(i)
    else:
        cat.append(i)


X1_train = X_train[num]
X1_test = X_test[num]

X2_train = X_train[cat]
X2_test = X_test[cat]

In [12]:
from itertools import combinations
from sklearn.preprocessing import PolynomialFeatures

def add_interactions(df1, df2):
    df = pd.concat([df1, df2])
    combos = list(combinations(list(df.columns), 2))
    colnames = list(df.columns) + ['_'.join(x) for x in combos]
    
    poly = PolynomialFeatures(interaction_only=True, include_bias=False)
    df = poly.fit_transform(df)
    df = pd.DataFrame(df)
    df.columns = colnames
    
    noint_indicies = [i for i, x in enumerate(list((df == 0).all())) if x]
    df = df.drop(df.columns[noint_indicies], axis = 1)
    return df[:len(df1)], df[len(df1):]

X1_train, X1_test = add_interactions(X1_train, X1_test)
X2_train, X2_test = add_interactions(X2_train, X2_test)

In [13]:
"""
import sklearn.feature_selection

select = sklearn.feature_selection.SelectKBest(k=len(X1_train.columns)//8)
selected_features = select.fit(X1_train, y_train)
indices_selected = selected_features.get_support(indices=True)
colnames_selected = [X1_train.columns[i] for i in indices_selected]

X1_train = X1_train[colnames_selected]
X1_test = X1_test[colnames_selected]

select = sklearn.feature_selection.SelectKBest(k=len(X2_train.columns)//8)
selected_features = select.fit(X2_train, y_train)
indices_selected = selected_features.get_support(indices=True)
colnames_selected = [X2_train.columns[i] for i in indices_selected]

X2_train = X2_train[colnames_selected]
X2_test = X2_test[colnames_selected]
"""

'\nimport sklearn.feature_selection\n\nselect = sklearn.feature_selection.SelectKBest(k=len(X1_train.columns)//8)\nselected_features = select.fit(X1_train, y_train)\nindices_selected = selected_features.get_support(indices=True)\ncolnames_selected = [X1_train.columns[i] for i in indices_selected]\n\nX1_train = X1_train[colnames_selected]\nX1_test = X1_test[colnames_selected]\n\nselect = sklearn.feature_selection.SelectKBest(k=len(X2_train.columns)//8)\nselected_features = select.fit(X2_train, y_train)\nindices_selected = selected_features.get_support(indices=True)\ncolnames_selected = [X2_train.columns[i] for i in indices_selected]\n\nX2_train = X2_train[colnames_selected]\nX2_test = X2_test[colnames_selected]\n'

In [14]:
from sklearn.decomposition import PCA
pca = PCA(n_components=min(len(pd.concat([X1_train, X1_test])), len(X1_train.columns)//8))
X_t = pd.DataFrame(pca.fit_transform(pd.concat([X1_train, X1_test])))
X1_train, X1_test = X_t[:len(X1_train)], X_t[len(X1_train):]

pca = PCA(n_components=len(X2_train.columns)//8)
X_t = pd.DataFrame(pca.fit_transform(pd.concat([X2_train, X2_test])))
X2_train, X2_test = X_t[:len(X2_train)], X_t[len(X2_train):]

In [15]:
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

In [ ]:
input_1 = tf.keras.Input(dtype = tf.float32, shape = (len(X1_train.columns),))
input_2 = tf.keras.Input(dtype = tf.float32, shape = (len(X2_train.columns),))

dense_layer_1_1 = tf.keras.layers.Dense(units = 64, activation = tf.nn.relu)(input_1)
dropout_1_5 = tf.keras.layers.Dropout(rate = 0.2)(dense_layer_1_1)


dense_layer_2_1 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(input_2)
dropout_2_5 = tf.keras.layers.Dropout(rate = 0.2)(dense_layer_2_1)

concat_layer = tf.keras.layers.Concatenate()([dropout_1_5, dropout_2_5])


dense_layer_3 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(concat_layer)
dropout_3_5 = tf.keras.layers.Dropout(rate = 0.8)(dense_layer_3)

#output = tf.keras.layers.Dense(units = 2, activation = tf.nn.softmax)(dense_layer_3)
output = tf.keras.layers.Dense(units = 2, activation = tf.nn.softmax)(dropout_3_5)

In [ ]:
model = tf.keras.Model(inputs=[input_1, input_2], outputs=output)

In [ ]:
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=tf.compat.v1.train.AdamOptimizer(learning_rate=0.00005), metrics=['acc'])
print(model.summary())

In [ ]:
#history = model.fit(x=X_train, y=y_train, batch_size=32, epochs=400, verbose=1, validation_split=0.1)
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=40)
history = model.fit(x=[X1_train, X2_train], y=y_train, batch_size=32, epochs=2000, verbose=1, validation_split=0.1, callbacks=[callback])

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])

plt.title('model accuracy')
plt.ylabel('acc')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()

In [ ]:
score = model.evaluate(x=[X1_test, X2_test], y=y_test, verbose=1)

print("Test Score:", score[0])
print("Test ACC:", score[1])

In [16]:
learning_rate = 0.00005
training_epochs = 200
batch_size = 32
num_models = 10

tf.compat.v1.disable_eager_execution()

class Model:

    def __init__(self, sess, name):
        self.sess = sess
        self.name = name
        self._build_net()

    def _build_net(self):
        with tf.compat.v1.variable_scope(self.name):
            
            self.training = tf.keras.backend.placeholder(dtype = tf.bool)

            self.X1 = tf.keras.backend.placeholder(dtype = tf.float32, shape=(None,len(X1_train.columns)))
            self.X2 = tf.keras.backend.placeholder(dtype = tf.float32, shape=(None,len(X2_train.columns)))            
            self.Y = tf.keras.backend.placeholder(dtype = tf.float32, shape=(None,2))

            dense1 = tf.compat.v1.layers.dense(inputs=self.X1, units = 64, activation=tf.nn.relu)
            dropout1 = tf.compat.v1.layers.dropout(inputs=dense1, rate=0.2, training=self.training)
            
            dense2 = tf.compat.v1.layers.dense(inputs=self.X2, units = 10, activation=tf.nn.relu)
            dropout2 = tf.compat.v1.layers.dropout(inputs=dense2, rate=0.2, training=self.training)
            
            concat_layer = tf.keras.layers.Concatenate()([dropout1, dropout2])

            dense3 = tf.compat.v1.layers.dense(inputs=concat_layer, units = 10, activation=tf.nn.relu)
            dropout3 = tf.compat.v1.layers.dropout(inputs=dense3, rate=0.8, training=self.training)
            
            
            self.logits = tf.keras.layers.Dense(units = 2, activation = tf.nn.softmax)(dropout3)

        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.logits, labels=self.Y))
        self.optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate).minimize(self.cost)

        correct_prediction = tf.equal(tf.argmax(self.logits, 1), tf.argmax(self.Y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    def predict(self, x1_test, x2_test, training=False):
        return self.sess.run(self.logits,
                             feed_dict={self.X1: x1_test, self.X2: x2_test, self.training: training})

    def get_accuracy(self, x1_test, x2_test, y_test, training=False):
        return self.sess.run(self.accuracy,
                             feed_dict={self.X1: x1_test, self.X2: x2_test,
                                        self.Y: y_test, self.training: training})

    def train(self, x1_data, x2_data, y_data, training=True):
        return self.sess.run([self.cost, self.optimizer], feed_dict={
            self.X1: x1_data, self.X2: x2_data, self.Y: y_data, self.training: training})

    
sess = tf.compat.v1.Session()


models = []
for m in range(num_models):
    models.append(Model(sess, "model" + str(m)))

sess.run(tf.compat.v1.global_variables_initializer())

print('Learning Started!')


for epoch in range(training_epochs):
    avg_cost_list = np.zeros(len(models))
    total_batch = int(len(X1_train)/ batch_size)
    for i in range(total_batch):
        batch_xs1, batch_xs2, batch_ys = X1_train[i * batch_size:(i+1) * batch_size], X2_train[i * batch_size:(i+1) * batch_size], y_train[i * batch_size:(i+1) * batch_size]
        for m_idx, m in enumerate(models):
            c, _ = m.train(batch_xs1, batch_xs2, batch_ys)
            avg_cost_list[m_idx] += c / total_batch
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', avg_cost_list)

print('Learning Finished!')


test_size = len(y_test)
predictions = np.zeros([test_size, 2])
for m_idx, m in enumerate(models):
    print(m_idx, 'Accuracy:', m.get_accuracy(
        X1_test, X2_test, y_test))
    p = m.predict(X1_test, X2_test) 
    predictions += p

ensemble_correct_prediction = tf.equal(tf.argmax(predictions, 1), tf.argmax(y_test, 1))
ensemble_accuracy = tf.reduce_mean(tf.cast(ensemble_correct_prediction,  tf.float32))
print('Ensemble accuracy:', sess.run(ensemble_accuracy))

#dir = os.path.dirname(os.path.realpath(__file__))
#all_saver = tf.train.Saver() 
#all_saver.save(sess, dir + '/0909.h5')

Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use keras.layers.dropout instead.
Learning Started!
Epoch: 0001 cost = [0.69290214 0.70077731 0.69969636 0.71201616 0.68392486 0.70386796
 0.71650612 0.69721202 0.70229658 0.70417356]
Epoch: 0002 cost = [0.68112946 0.68748743 0.68469867 0.69597056 0.6707163  0.68537659
 0.69408024 0.68714272 0.68814823 0.68721747]
Epoch: 0003 cost = [0.66913488 0.67518549 0.67226866 0.68293151 0.66150814 0.67230107
 0.67572839 0.680974   0.67762983 0.67205154]
Epoch: 0004 cost = [0.65751194 0.66615875 0.66399635 0.66763701 0.6550242  0.6598266
 0.66729791 0.67268445 0.67310835 0.66333692]
Epoch: 0005 cost = [0.64880452 0.65721556 0.65460837 0.65973745 0.6462252  0.64901053
 0.65331925 0.66585299 0.66477225 0.65038127]
Epoch: 0006 cost = [0.64580229 0.64978614 0.6418056

Epoch: 0049 cost = [0.53398887 0.53348193 0.54029558 0.53544319 0.5382489  0.53403349
 0.53177598 0.53789038 0.54949732 0.53238617]
Epoch: 0050 cost = [0.53567303 0.53387713 0.53386273 0.53697107 0.53557947 0.53074154
 0.52628286 0.53406516 0.54452564 0.53500682]
Epoch: 0051 cost = [0.5315575  0.53130186 0.53539332 0.52984133 0.53129887 0.52975204
 0.52704829 0.53423632 0.54224601 0.53102097]
Epoch: 0052 cost = [0.53331183 0.52576164 0.53227623 0.53055986 0.53255547 0.52362634
 0.5216162  0.5350201  0.54072423 0.53186964]
Epoch: 0053 cost = [0.5285862  0.52961124 0.53154158 0.53412167 0.53070401 0.52638137
 0.5232323  0.53146497 0.54332202 0.52857869]
Epoch: 0054 cost = [0.53043178 0.5277407  0.53262965 0.52691561 0.52739856 0.52428138
 0.5177243  0.53009793 0.53815647 0.522832  ]
Epoch: 0055 cost = [0.52352408 0.52604633 0.52978847 0.53269976 0.52880425 0.52249611
 0.52067244 0.52757865 0.53543986 0.52118501]
Epoch: 0056 cost = [0.52797451 0.52374834 0.52860888 0.52489396 0.52178697 0

Epoch: 0112 cost = [0.47896437 0.47210181 0.47801108 0.47307933 0.46167362 0.46537473
 0.46967098 0.47874144 0.47247725 0.47303509]
Epoch: 0113 cost = [0.47669476 0.47386489 0.47643104 0.47305562 0.45919205 0.46534093
 0.47116318 0.47777431 0.47480772 0.47510023]
Epoch: 0114 cost = [0.47813607 0.47374498 0.48017322 0.47543628 0.4581125  0.46772613
 0.46947149 0.479521   0.47698351 0.4786892 ]
Epoch: 0115 cost = [0.48094335 0.47104852 0.47614996 0.47171223 0.46234321 0.46646959
 0.46948045 0.47628488 0.46963535 0.47266435]
Epoch: 0116 cost = [0.47413409 0.47304308 0.47582968 0.47364502 0.45805873 0.46408316
 0.46727002 0.47898649 0.47285338 0.47008379]
Epoch: 0117 cost = [0.47487624 0.47228101 0.4721834  0.47073113 0.45139172 0.46401695
 0.46639367 0.47590913 0.46942835 0.47447557]
Epoch: 0118 cost = [0.47289527 0.47185367 0.47547312 0.46883969 0.45770253 0.4605684
 0.47023912 0.47382017 0.47090737 0.46994583]
Epoch: 0119 cost = [0.47507383 0.46735929 0.4719259  0.47256837 0.46041064 0.

Epoch: 0175 cost = [0.45800869 0.45557434 0.45869373 0.44926304 0.43802626 0.44250137
 0.4531522  0.46149645 0.44784778 0.45498108]
Epoch: 0176 cost = [0.46016828 0.45498525 0.45838836 0.45479559 0.42726159 0.44099867
 0.45262205 0.45970168 0.4496265  0.45521011]
Epoch: 0177 cost = [0.45735168 0.45703485 0.45917525 0.45239103 0.43490089 0.43970273
 0.45178668 0.45835353 0.44431401 0.44744707]
Epoch: 0178 cost = [0.459879   0.45466807 0.45657056 0.45172805 0.43122456 0.44561883
 0.45261957 0.46112518 0.44358116 0.45136885]
Epoch: 0179 cost = [0.46185442 0.45287004 0.4616524  0.45056566 0.42431782 0.43915604
 0.45216425 0.46061983 0.44668503 0.45274072]
Epoch: 0180 cost = [0.46002318 0.45439432 0.46089127 0.4513991  0.43345657 0.437619
 0.45613375 0.45709702 0.44815903 0.45228192]
Epoch: 0181 cost = [0.45823601 0.45351975 0.45716876 0.45363747 0.43297109 0.43989629
 0.4493969  0.45845696 0.44836093 0.45118344]
Epoch: 0182 cost = [0.45738608 0.44967527 0.46224171 0.45220947 0.42739071 0.4